In [2]:
#packages
import numpy
import pandas as pd
import os
from riotwatcher import LolWatcher, ApiError

In [3]:
#need to put this into a class
lol_watcher = LolWatcher('RGAPI-62ebc116-6031-43aa-b51d-08c17a5eada6') #api key

In [4]:
#####################
## Players 
#####################

def get_Player(region, name):
    
    valid_region_names = ['oc1', 'na1', 'eun1']
    if region in valid_region_names:
        player = lol_watcher.summoner.by_name(region, name.lower())
        return lol_watcher.league.by_summoner('oc1', player['id'])
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')    

#####################
## Patches 
#####################
        
def get_CurrentPatchVersions(region):
    
    valid_region_names = ['oce', 'na1', 'eune']
    if region in valid_region_names:
        return lol_watcher.data_dragon.versions_for_region(region)
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')   

def get_AllPatchVersions():
    
    return [i for i in lol_watcher.data_dragon.versions_all() if not i.startswith('lolpatch')]

#####################
## Champions
#####################

def get_Champions(patchversion = 'latest'):
    
    if patchversion == 'latest':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in [get_AllPatchVersions()[0]]] #nested by patch        
    elif patchversion == 'all':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in get_AllPatchVersions()] #nested by patch
    else:
        champions = [[(patch, champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in patchversion] #nested by patch
        
    return pd.DataFrame([val for sublist in champions for val in sublist], columns = ['patchversion', 'champion']) #compress into single list

def get_ChampionStats(champions):
        data = pd.DataFrame.from_dict([{
                'patch': patchversion, 
                'champion': champion, 
                'championtype': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['tags'],
                'spellresource': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['partype'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['info'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']} 
            for patchversion, champion in champions.values.tolist()
        ])
        
        #create championtype
        return pd.concat([data, pd.get_dummies(data['championtype'].apply(pd.Series).stack(), prefix = 'championtype').sum(level=0)], axis = 1).drop(columns=['championtype'])

#####################
## Champion Abilities
#####################    

#riot api doesnt publish this. can use the below sights to webscrape the abilies and version history.
#however the format isnt desirable
#https://leagueoflegends.fandom.com/wiki/Annie/LoL
#https://leagueoflegends.fandom.com/wiki/Annie/LoL/Patch_history

#####################
## Items
#####################    

def get_Items(patchversion):
    data = pd.DataFrame.from_dict({
                    **lol_watcher.data_dragon.items(patchversion)['data']
            }).drop(
                index=['description', 'colloq', 'plaintext', 'into', 'image', 'from', 'depth',
                   'inStore', 'effect', 'consumed', 'stacks', 'hideFromAll',
                   'consumeOnFull', 'specialRecipe', 'requiredChampion', 'requiredAlly'
                ]).transpose()


    return pd.concat([
                data.name,
                data.gold.apply(lambda x: pd.Series(x) if isinstance(x, dict) else x).add_prefix('gold_'),
                pd.get_dummies(data.tags.apply(pd.Series).stack(), prefix='tags').sum(level=0),
                data.maps.apply(lambda x: pd.Series(x) if isinstance(x, dict) else x).add_prefix('maps_'),
                data.stats.apply(lambda x: pd.Series(x) if isinstance(x, dict) else x).add_prefix('stats_'),
            ], axis = 1)

#####################
## Summoner Spells
#####################    
#will need to do extra work to get other stats from description such as unleashed tp, ignite dmg and exhaust dmg mitigated

def get_SummonerSpells(patchversion):

    return pd.DataFrame([[  
                lol_watcher.data_dragon.summoner_spells(patchversion)['data'][spell]['name'],
                lol_watcher.data_dragon.summoner_spells(patchversion)['data'][spell]['cooldown'][0],
                lol_watcher.data_dragon.summoner_spells(patchversion)['data'][spell]['range'][0],
                lol_watcher.data_dragon.summoner_spells(patchversion)['data'][spell]['modes']
             ]
             for spell in  lol_watcher.data_dragon.summoner_spells(patchversion)['data']
            ], columns = ['name', 'cooldown', 'range', 'modes']
    )

#####################
## Runes
#####################  

#lol_watcher.data_dragon.runes_reforged('12.22.1')

#####################
## Game/Match
##################### 

#lol_watcher.match.by_id()

{'type': 'item',
 'version': '12.22.1',
 'basic': {'name': '',
  'rune': {'isrune': False, 'tier': 1, 'type': 'red'},
  'gold': {'base': 0, 'total': 0, 'sell': 0, 'purchasable': False},
  'group': '',
  'description': '',
  'colloq': ';',
  'plaintext': '',
  'consumed': False,
  'stacks': 1,
  'depth': 1,
  'consumeOnFull': False,
  'from': [],
  'into': [],
  'specialRecipe': 0,
  'inStore': True,
  'hideFromAll': False,
  'requiredChampion': '',
  'requiredAlly': '',
  'stats': {'FlatHPPoolMod': 0,
   'rFlatHPModPerLevel': 0,
   'FlatMPPoolMod': 0,
   'rFlatMPModPerLevel': 0,
   'PercentHPPoolMod': 0,
   'PercentMPPoolMod': 0,
   'FlatHPRegenMod': 0,
   'rFlatHPRegenModPerLevel': 0,
   'PercentHPRegenMod': 0,
   'FlatMPRegenMod': 0,
   'rFlatMPRegenModPerLevel': 0,
   'PercentMPRegenMod': 0,
   'FlatArmorMod': 0,
   'rFlatArmorModPerLevel': 0,
   'PercentArmorMod': 0,
   'rFlatArmorPenetrationMod': 0,
   'rFlatArmorPenetrationModPerLevel': 0,
   'rPercentArmorPenetrationMod': 0,
   

# Data